In [1]:
import warnings
warnings.filterwarnings('ignore')

import os, sys

print(os.getcwd())
sys.path.append(os.getcwd())

/mnt/d/Dropbox/Ranee_Joshi_PhD_Local/04_PythonCodes/dh2loop/notebooks


In [2]:
sys.path

['/home/ranee/anaconda3/envs/dh2loop/lib/python36.zip',
 '/home/ranee/anaconda3/envs/dh2loop/lib/python3.6',
 '/home/ranee/anaconda3/envs/dh2loop/lib/python3.6/lib-dynload',
 '',
 '/home/ranee/.local/lib/python3.6/site-packages',
 '/home/ranee/.local/lib/python3.6/site-packages/map2loop-0.0.50-py3.6.egg',
 '/home/ranee/.local/lib/python3.6/site-packages/LoopStructural-0.0.0-py3.6-linux-x86_64.egg',
 '/home/ranee/.local/lib/python3.6/site-packages/dh2loop-0.0.1-py3.6.egg',
 '/home/ranee/anaconda3/envs/dh2loop/lib/python3.6/site-packages',
 '/home/ranee/anaconda3/envs/dh2loop/lib/python3.6/site-packages/IPython/extensions',
 '/tmp/tmpsz1jlk0u',
 '/mnt/d/Dropbox/Ranee_Joshi_PhD_Local/04_PythonCodes/dh2loop/notebooks']

In [3]:
import pandas as pd
from dh2loop import dh2l

In [4]:
%run -i "dh2l_config.py"

Default parameters loaded from dh2l_config.py:
#Database Files
data_path='../data/'
wamex_path='../data/wamex/'
collar_file=wamex_path+'collar.csv'
collarattr_file=wamex_path+'collarattr.csv'
dhsurvey_file=wamex_path+'dhsurvey.csv'
dhsurveyattr_file=wamex_path+'dhsurveyattr.csv'
dhgeology_file=wamex_path+'dhgeology.csv'
dhgeologyattr_file=wamex_path+'dhgeologyattr.csv'

#Thesauri
rl_maxdepth_dic_file= wamex_path+'rl_maxdepth_dic.csv'
survey_dic_file=wamex_path+'survey_dic.csv'
dic_attr_col_lithology_file=wamex_path+'dic_att_col_lithology.csv'
dic_attr_val_lithology_file=wamex_path+'dic_attr_val_lithology_filter.csv'
cleanup_lithology_file=wamex_path+'cleanup_lithology.csv'
litho_dic_file=wamex_path+'litho_dic_1.csv'
CET_hierarchy_dico_file=wamex_path+'hierarchy_dico.csv'

#ExportFiles
export_path='../data/export/'
DB_Collar_Export=export_path+'DB_Collar_Export.csv'
DB_Survey_Export=export_path+'DB_Survey_Export.csv'
DB_Survey_Export_Calc=export_path+'DB_Survey_Export_Calc.csv'
CET_Lith

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import numpy as np
from glove import Glove
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords
import json
import h5py

[nltk_data] Downloading package stopwords to /home/ranee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ranee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [6]:
def load_geovec(path):
    instance = Glove()
    with h5py.File(path, 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)
    instance.word_vectors = v
    instance.no_components = v.shape[1]
    instance.word_biases = np.zeros(v.shape[0])
    instance.add_dictionary(dct)
    return instance

new_model = load_geovec('../glove/geovec_300d_v1.h5')
#modelEmb = Glove.load('/home/ignacio/Documents/chapter2/best_glove_300_317413_w10_lemma.pkl')

In [14]:
#Stopwords
extra_stopwords = [
    'also',
]
stop = stopwords.words('english') + extra_stopwords


def tokenize(text, min_len=1):
    '''Function that tokenize a set of strings
    Input:
        -text: set of strings
        -min_len: tokens length
    Output:
        -list containing set of tokens'''

    tokens = [word.lower() for sent in nltk.sent_tokenize(text.read())
              for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if token.isalpha() and len(token) >= min_len:
            filtered_tokens.append(token)

    return [x.lower() for x in filtered_tokens if x not in stop]


def tokenize_and_lemma(text, min_len=0):
    '''Function that retrieves lemmatised tokens
    Inputs:
        -text: set of strings
        -min_len: length of text
    Outputs:
        -list containing lemmatised tokens'''
    filtered_tokens = tokenize(text, min_len=min_len)

    lemmas = [lemma.lemmatize(t) for t in filtered_tokens]
    return lemmas

In [23]:
test = open('../data/testdata.csv', 'r')
tokenize_list=tokenize(test, min_len=1)
tokenize_and_lemma_list=tokenize_and_lemma(test, min_len=1)
print(tokenize_list)

['nso', 'woo', 'mao', 'qtg', 'lc', 'ls', 'lfp', 'mvb', 'v', 'fio', 'tl', 'mb', 'qa', 'cza', 'czlp', 'ash', 'ashg', 'qahp', 'au', 'auk', 'aux', 'cztc', 'ag', 'vq', 'ab', 'lat', 'lsp', 'tcs', 'czlu', 'czlc', 'au', 'czz', 'czf', 'x', 'ash', 'qas', 'ab', 'ac', 'czc', 'ferricrete', 'calcrete', 'dolomite', 'clay', 'col', 'lnd', 'lmz', 'fgr', 'aut', 'x', 'aod', 'liz', 'lpz', 'lcz', 'mba', 'ta', 'lf', 'iv', 'ivs', 'scp', 'sh', 'fip', 'sc', 'mvm', 'mv', 'vcb', 'ivx', 'qh', 'qts', 'qtc', 'qtg', 'qsil', 'qtp', 'lfn', 'vqz', 'lm', 'lc', 'ai', 'czl', 'tlm', 'qdh', 'sch', 'tls', 'qap', 'qdc', 'tlc', 'mga', 'lsz', 'qv', 'cy', 'us', 'tg', 'vs', 'spt', 'sif', 'ssh', 'sct', 'utc', 'soil', 'grt', 'cz', 'mrk', 'cc', 'mm', 'nsr', 'f', 'chs', 'md', 'gvl', 'sst', 'cyt', 'shl', 'af', 'czz', 'czq', 'afp', 'abf', 'void', 'agr', 'amig', 'amm', 'fault', 'auc', 'avn', 'qas', 'ques', 'abt', 'aci', 'czf', 'ass', 'ush', 'rfo', 'nfo', 'u', 'nr', 'sp', 'sps', 'sx', 'eo', 'ccf', 'msc', 'ssq', 'lfz', 'tcg', 'tcc', 'lcz',

In [18]:
def get_vector(word, model, return_zero=False):
    '''Function that retrieves word embeddings (vector)
    Inputs:
        -word: token (string)
        -model: trained MLP model
        -return_zero: boolean variable
    Outputs:
        -wv: numpy array (vector)'''
    epsilon = 1.e-10

    unk_idx = model.dictionary['unk']
    idx = model.dictionary.get(word, unk_idx)
    wv = model.word_vectors[idx].copy()

    if return_zero and word not in model.dictionary:
        n_comp = model.word_vectors.shape[1]
        wv = np.zeros(n_comp) + epsilon

    return wv

In [ ]:
def mean_embeddings(dataframe_file, model):
    '''Function to retrieve sentence embeddings from dataframe with
    lithological descriptions.
    Inputs:
        -dataframe_file: pandas dataframe containing lithological descriptions
                         and reclassified lithologies
        -model: word embeddings model generated using GloVe
    Outputs:
        -DF: pandas dataframe including sentence embeddings'''
    DF = pd.read_pickle(dataframe_file)
    DF = DF.drop_duplicates(subset=['x', 'y', 'z'])
    DF['tokens'] = DF['Description'].apply(lambda x: tokenize_and_lemma(x))
    DF['length'] = DF['tokens'].apply(lambda x: len(x))
    DF = DF[DF.length > 0]
    DF['vectors'] = DF['tokens'].apply(lambda x: np.asarray([get_vector(n, model) for n in x]))
    DF['mean'] = DF['vectors'].apply(lambda x: np.mean(x[~np.all(x == 1.e-10, axis=1)], axis=0))
    DF['reclass'] = pd.Categorical(DF.reclass)
    DF['code'] = DF.reclass.cat.codes
    DF['drop'] = DF['mean'].apply(lambda x: (~np.isnan(x).any()))
    DF = DF[DF['drop']]
    return DF

In [ ]:
def split_stratified_dataset(Dataframe, test_size, validation_size):
    '''Function that split dataset into test, training and validation subsets
    Inputs:
        -Dataframe: pandas dataframe with sentence mean_embeddings
        -test_size: decimal number to generate the test subset
        -validation_size: decimal number to generate the validation subset
    Outputs:
        -X: numpy array with embeddings
        -Y: numpy array with lithological classes
        -X_test: numpy array with embeddings for test subset
        -Y_test: numpy array with lithological classes for test subset
        -Xt: numpy array with embeddings for training subset
        -yt: numpy array with lithological classes for training subset
        -Xv: numpy array with embeddings for validation subset
        -yv: numpy array with lithological classes for validation subset
        '''
    X = np.vstack(Dataframe['mean'].values)
    Y = Dataframe.code.values.reshape(len(Dataframe.code), 1)
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        Y,
                                                        test_size=test_size,
                                                        stratify=Y,
                                                        random_state=42)
    Xt, Xv, yt, yv = train_test_split(X_train,
                                      y_train,
                                      test_size=validation_size,
                                      stratify=None,
                                      random_state=1)
    return X, Y, X_test, y_test, Xt, yt, Xv, yv


def retrieve_predictions(classifier, x):
    '''Function that retrieves lithological classes using the trained classifier
    Inputs:
        -classifier: trained MLP classifier
        -x: numpy array containing embbedings
    Outputs:
        -codes_pred: numpy array containing lithological classes predicted'''
    preds = classifier.predict(x, verbose=0)
    new_onehot = np.zeros((x.shape[0], 18))
    new_onehot[np.arange(len(preds)), preds.argmax(axis=1)] = 1
    codes_pred = one_enc.inverse_transform(new_onehot)
    return codes_pred


def classifier_assess(classifier, x, y):
    '''Function that prints the performance of the classifier
    Inputs:
        -classifier: trained MLP classifier
        -x: numpy array with embeddings
        -y: numpy array with lithological classes predicted'''
    Y2 = retrieve_predictions(classifier, x)
    print('f1 score: ', metrics.f1_score(y, Y2, average='macro'),
          'accuracy: ', metrics.accuracy_score(y, Y2),
          'balanced_accuracy:', metrics.balanced_accuracy_score(y, Y2))


def save_predictions(Dataframe, classifier, x, name):
    '''Function that saves dataframe predictions as a pickle file
    Inputs:
        -Dataframe: pandas dataframe with mean_embeddings
        -classifier: trained MLP model,
        -x: numpy array with embeddings,
        -name: string name to save dataframe
    Outputs:
        -save dataframe'''
    preds = classifier.predict(x, verbose=0)
    Dataframe['predicted_probabilities'] = preds.tolist()
    Dataframe['pred'] = retrieve_predictions(classifier, x).astype(np.int32)
    Dataframe[['x', 'y', 'FromDepth', 'ToDepth', 'TopElev', 'BottomElev',
               'mean', 'predicted_probabilities', 'pred', 'reclass', 'code']].to_pickle('{}.pkl'.format(name))

In [ ]:
# loading word embeddings model
# (This can be obtained from https://github.com/spadarian/GeoVec )
modelEmb = Glove.load('/home/ignacio/Documents/chapter2/best_glove_300_317413_w10_lemma.pkl')

# getting the mean embeddings of descriptions
DF = mean_embeddings('manualTest.pkl', modelEmb)

# subseting dataset for training classifier
X, Y, X_test, Y_test, X_train, Y_train, X_validation, Y_validation = split_stratified_dataset(DF, 0.1, 0.1)

# encoding lithological classes
encodes = one_enc.fit_transform(Y_train).toarray()

# MLP model generation
model = Sequential()
model.add(Dense(100, input_dim=300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(units=len(DF.code.unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# training MLP model
model.fit(X_train, encodes, epochs=30, batch_size=100, verbose=2)

# saving MLP model
model.save('mlp_prob_model.h5')

# assessment of model performance
classifier_assess(model, X_validation, Y_validation)

# save lithological prediction likelihoods dataframe
save_predictions(DF, model, X, 'NSWpredictions')

## Exporting Collar Data

In [12]:
collar= pd.read_csv(collar_file,encoding = "ISO-8859-1", dtype='object')
collarattr= pd.read_csv(collarattr_file,encoding = "ISO-8859-1", dtype='object')
rl_maxdepth_dic= pd.read_csv(rl_maxdepth_dic_file,encoding = "ISO-8859-1", dtype='object')
dh2l.collar_attr_col_dic(rl_maxdepth_dic_file)
dh2l.collar_collarattr_final(collar_file, collarattr_file, rl_maxdepth_dic_file, DB_Collar_Export)
DB_Collar_Export_table=pd.read_csv(DB_Collar_Export,encoding = "ISO-8859-1", dtype='object')
DB_Collar_Export_table= DB_Collar_Export_table.loc[:, ~DB_Collar_Export_table.columns.str.contains('^Unnamed')]
DB_Collar_Export_table[['X','Y']] = DB_Collar_Export_table.apply(dh2l.convert_coords,axis=1)
DB_Collar_Export_table.to_csv(DB_Collar_Export)
print(DB_Collar_Export_table)

    CollarID                    HoleId Longitude  Latitude       RL MaxDepth  \
0    1146528  71288Minjar NorthSSDD008   116.947  -28.9021  356.255    546.4   
1    1146530  71288Minjar NorthSSDD010   116.949  -28.9083  356.871    405.0   
2    1146531  71288Minjar NorthSSDD011   116.948  -28.9002  360.214    249.0   
3    1146650  71288Minjar NorthSSDD005   116.948  -28.9002  357.153    468.0   
4    1233095        82655KararaBHN1001   116.818  -29.1504  357.156      150   
..       ...                       ...       ...       ...      ...      ...   
127   931216         79530KararaLSC016   117.148   -29.426  315.535      124   
128   931217         79530KararaLSC017   117.144  -29.4045  361.603      158   
129   931218         79530KararaLSC018   117.151  -29.4335  323.052      136   
130   931219         79530KararaLSD006   117.153  -29.4336  365.334      248   
131   931220         79530KararaLSD007   117.151  -29.4405  368.845    260.7   

                 X             Y  
0   

## Exporting Survey Data

In [13]:
dhsurvey= pd.read_csv(dhsurvey_file,encoding = "ISO-8859-1", dtype='object')
dhsurveyattr= pd.read_csv(dhsurveyattr_file,encoding = "ISO-8859-1", dtype='object')
survey_dic= pd.read_csv(survey_dic_file,encoding = "ISO-8859-1", dtype='object')
dh2l.survey_attr_col_dic(survey_dic_file)
dh2l.survey_final(dhsurvey_file,dhsurveyattr_file, DB_Survey_Export)
dhsurvey= pd.read_csv(DB_Survey_Export,encoding = "ISO-8859-1", dtype='object')
dhsurvey= dhsurvey.loc[:, ~dhsurvey.columns.str.contains('^Unnamed')]
dh2l.convert_survey(DB_Collar_Export,DB_Survey_Export, DB_Survey_Export_Calc)
dhsurvey_calc= pd.read_csv(DB_Survey_Export_Calc,encoding = "ISO-8859-1", dtype='object')
dhsurvey_calc.dropna()
dhsurvey_calc= dhsurvey_calc.loc[:, ~dhsurvey_calc.columns.str.contains('^Unnamed')]
print(dhsurvey_calc)

     CollarID       Depth     Azimuth         Dip Index              X  \
0      153802    0.000000   90.000000  -50.000000     2  495028.106029   
1      153802    0.000000   90.000000  -50.000000     2  494917.618977   
2      153803    0.000000   90.000000  -55.000000     4  495028.106029   
3      153803    0.000000   90.000000  -55.000000     4  494906.041567   
4      153804    0.000000   90.000000  -60.000000     6  495028.106029   
...       ...         ...         ...         ...   ...            ...   
1345  1233228  140.000000  137.400000  -90.000000  1339  482677.460027   
1346  1233228  160.000000  137.800000  -90.000000  1339  482663.948285   
1347  1233228  180.000000  137.200000  -90.000000  1339  482650.372272   
1348  1233228  200.000000  137.300000  -90.000000  1339  482636.821912   
1349  1233228  220.000000  137.400000  -90.000000  1339  482630.053152   

                   Y           Z  
0     6802449.793954  390.166000  
1     6802449.793954  349.952002  
2     

## Exporting Lithology Data

In [ ]:
dhgeology= pd.read_csv(dhgeology_file,encoding = "ISO-8859-1", dtype='object')
dhgeologyattr= pd.read_csv(dhgeologyattr_file,encoding = "ISO-8859-1", dtype='object')
dh2l.litho_attr_val_dic(dic_attr_val_lithology_file)
dh2l.litho_dico (litho_dic_file)
dh2l.clean_up (cleanup_lithology_file)
dh2l.litho_attr_val_with_fuzzy (CET_Litho)
dh2l.litho_final (collar_file, dhgeology_file, dhgeologyattr_file, dic_attr_col_lithology_file, CET_Litho, DB_Lithology_Export)
dic_att_col_lithology=pd.read_csv(dic_attr_col_lithology_file,encoding = "ISO-8859-1", dtype='object')
dic_attr_val_lithology=pd.read_csv(dic_attr_val_lithology_file,encoding = "ISO-8859-1", dtype='object')
cleanup_lithology=pd.read_csv(cleanup_lithology_file,encoding = "ISO-8859-1", dtype='object')
lithology_thesaurus=pd.read_csv(litho_dic_file,encoding = "ISO-8859-1", dtype='object')
DB_Lithology_Export_View=pd.read_csv(DB_Lithology_Export,encoding = "ISO-8859-1", dtype='object')
print(DB_Lithology_Export_View)

## Upscaling Lithology Data

In order to link and upscale the drillhole information, a <b> hierarchical thesaurus (Thesaurus 4) </b> was also built. The dictionary involved three levels that would upscale a list of 757 rock names to more general rock groups.

In [ ]:
CET_hierarchy_dico = pd.read_csv(CET_hierarchy_dico_file,encoding = "ISO-8859-1")

In [ ]:
dh2l.upscale_litho (DB_Lithology_Export, CET_hierarchy_dico_file, DB_Lithology_Upscaled)
Upscaled_Litho= pd.read_csv(DB_Lithology_Upscaled,encoding = "ISO-8859-1")
Upscaled_Litho.head()

In [ ]:
Upscaled_Litho[Upscaled_Litho['Level_1'] == "igneous"]

In [ ]:
Upscaled_Litho[Upscaled_Litho['Level_2'] == "ore"]

In [ ]:
Upscaled_Litho[Upscaled_Litho['CET_Litho'] == "granodiorite"]

Let's also calculate for the X,Y,Z coordinates for FromDepth, ToDepth and its midpoint.

In [ ]:
dh2l.convert_lithology(DB_Collar_Export, DB_Survey_Export, DB_Lithology_Upscaled, DB_Lithology_Export_Calc)
DB_Lithology_Export_Calculated=pd.read_csv(DB_Lithology_Export_Calc)
print(DB_Lithology_Export_Calculated)

## Export to VTK

In [ ]:
dh2l.intervals2vtk(DB_Lithology_Export_Calc, DB_Lithology_Export_VTK)

<img src='./graphics/vtk.png'>

Open Paraview to view the VTK files and have some fun!<br>
The VTK file output is at \dh2loop\data\export\DB_Lithology_Export.vtp